In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

#The stars
import tensorflow as tf
from tensorflow import keras

In [ ]:
tf.__version__

In [ ]:
keras.__version__

In [ ]:
tf.config.list_physical_devices()

# Datos: Clasificacion de ropa (Fashion_MNIST)

Asi como sklearn trae utilidades para cargar datasets estandar, keras también trae. En general, keras puede aceptar datasets en forma de Numpy Arrays (como sklearn), pero también trae una clase Dataset que esta optimizada para cargar datos (incluso si son mas grandes que la memoria RAM del equipo).

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

Veamos el shape y el tipo de datos:

In [ ]:
print(X_train_full.shape,y_train_full.shape)
print(X_train_full.dtype)

Tenemos la imagen en (28,28) y cada pixel es un entero.

En Datasets grandes, CrossValidation puede ser muy costoso, por lo cual se recomienda separar un conjunton de validación aparte del training. Esto hacemos a continuación.

También normalizamos los píxeles (que van de 0 a 255) para que estén entre 0 y 1.

In [ ]:
#Separo en entrenamiento y validacion, y normalizo los pixeles
X_valid, X_train = X_train_full[:5000] / 255., X_train_full[5000:] / 255.
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_test = X_test / 255.

In [ ]:
print(X_train_full.shape)

plt.imshow(X_train[0], cmap='binary', interpolation='bicubic')

Los targets son numericos, del 0 al 9. Podemos guardar las etiquetas asi nos es mas facil analizar que tan bien o mal funciona nuestro modelo:

In [ ]:
class_names = ["Remera/top", "Pantalones", "Pullover", "Vestido", "Abrigo",
               "Sandalias", "Camisa", "Zapatillas", "Bolso", "Bota"]

n_rows = 4
n_cols = 10
plt.figure(figsize=(n_cols * 1.2, n_rows * 1.2))
for row in range(n_rows):
    for col in range(n_cols):
        index = n_cols * row + col
        plt.subplot(n_rows, n_cols, index + 1)
        plt.imshow(X_train[index], cmap="binary", interpolation="nearest")
        plt.axis('off')
        plt.title(class_names[y_train[index]], fontsize=12)
plt.subplots_adjust(wspace=0.2, hspace=0.5)
plt.show()

# Definir un modelo

Hay tres formas de definir modelos:
* Una es utilizando la clase de ``keras.models.Sequential``, que es apta para modelos donde solamente Layers adyacentes se encuentran conectados entre sí. 
* Otra es API funcional, que provee mucha mas flexibilidad, ya que se puede construir cualquier clase de red dirigida acíclica. 
* La ultima forma, es con SubClassing, es decir definiendo nuestras propias clases que hereden de las definidas en Keras, lo cual nos permite extender Keras para nuestras necesidades específicas, aunque no es recomendable a menos que realmente sepamos lo que estamos haciendo.

Si bien ahora quizás les sea mas sencillo utilizar la API secuencial, aprender a utilizar la API funcional es casi igual de fácil y provee mas flexibilidad, asi que es la forma que recomiendo de aprender a usar.

## API Secuencial

### Modo 1

Podemos crear un modelo, e ir añadiendo Layers sobre la marcha.

In [ ]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
#Creo un modelo secuencial
model = keras.models.Sequential()
#Y le voy agregando layers
model.add(keras.layers.Flatten(input_shape=[28, 28])) #matriz->vector (como np.ravel)
model.add(keras.layers.Dense(300, activation="relu")) #hidden 1
model.add(keras.layers.Dense(100, activation="relu")) #hidden 2
model.add(keras.layers.Dense(10, activation="softmax")) #output

Las funciones de activacion son muy importantes a la hora de usar DNN. Son las que agregan no-linearidad a la situacion.

En las capas internas, la mas usual es ReLU, una funcion que es 0 si $z\leq0$ y z si $z>0$. Hay otras opciones que suelen ser mejores, en particular para datasets grandes y con ciertas condiciones, pero ReLU tiene mucha ventaja en velocidad y optimizacion de librerias.

En la capa externa, va a depender del problema.


*   Para clasificacion, se utilizan sigmoide o softmax.
*   Para regresion, si no tenemos ninguna restriccion no utilizamos ninguna funcion de activacion. Si queremos $y\geq 0 $, podemos usar una funcion ReLu o una version mas suavizada llamada Softplus. Si queremos $a\leq y \leq b$, se puede utilizar una funcion sigmoide o tangente hiperbolica y re-escalearla.



In [ ]:
[m for m in dir(keras.activations) if not m.startswith("_")]

### Modo 2

También podemos directamente pasar la lista de Layers al constructor del Modelo ``Sequential``.

In [ ]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
#Sino directamente le paso una lista al crearlo:

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])

## API Funcional

La API funcional funciona definiendo Layers y utilizandolo como si fueran "funciones" que se aplican a los outputs de otros Layers. Keras irá rastreando estas conecciones, y luego generará un modelo entre el Layer de Inputs, y el de Outputs.

In [ ]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
input_ = keras.layers.Input(shape=[28, 28]) #En este caso es necesario definir el Layer de Inputs
flatten = keras.layers.Flatten()(input_)
hidden1 = keras.layers.Dense(300, activation="relu")(flatten)
hidden2 = keras.layers.Dense(100, activation="relu")(hidden1)
output = keras.layers.Dense(10, activation="softmax")(hidden2)#charlemos un poquito la softmax despues
model = keras.models.Model(inputs=[input_], outputs=[output])

# Compilar y visualizar

Una vez que hemos definido nuestro modelo, es necesario que lo "compilemos", entonces Keras creará el gráfico computacional en TensorFlow de acuerdo a como lo hemos definido. Al compilar necesitamos especificar la funcion de perdida, el optimizador y las distintas metricas que querramos almacenar.

In [ ]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])

La funcion de perdida define el problema y esta intimamente relacionada con la funcion de activacion de la output layer. Para clasificacion, tenemos algunas sutilezas.

Si tenemos un problema de dos clases, tenemos dos opciones:

* Utilizamos una unica neurona en la output layer. Esa neurona modela la asignacion a clase 1 o clase 2. La funcion de activacion sera entonces una sigmoide y la funcion de perdida la binary_cross_entropy.
* Utilizamos dos neuronas en la output layer. Cada una modela la probabilidad de pertenecer a una determinada clase. Dado que deben sumar 1, utilizamos la funcion de activacion softmax.  La funcion de perdida sera categorica. Pero aun hay mas. Si utilizamos la representacion $y={0,1}$, utilizamos sparse_categorical_cross_entropy. Si utilizamos $y={[0,1],[1,0]}$, utilizamos simplemente categorical_cross_entropy. Se puede pasar de sparse a categorical con `tensorflow.keras.utils.to_categorical` y de categorical a sparse con `np.argmin()`.

Cuando tenemos N clases, tenemos lo siguiente:

*   Si tenemos un problema MultiClase, en el que queremos asignar la medicion a una de las clases, utilizamos funcion de activacion softmax y categorical_cross_entropy o sparse_categorical_cross_entropy dependiendo de como sea $y$.
*   Si tenemos N clasificaciones en donde solo queremos decir si pertenece o no a determinada clase, utilizamos funcion de activacion sigmoide y cateogircal_cross_entropy (con el one hot encoding).






Podemos acceder a los diferentes Layers, mediante el atributo ``.layers``

In [ ]:
model.layers

Y También podemos imprimir un resumen de nuestro modelo, asi como un lindo grafico de el:

In [ ]:
model.summary()

In [ ]:
keras.utils.plot_model(model, "my_fashion_mnist_model.png", show_shapes=True)

Cada layer tiene atributos a los que podemos acceder, como el nombre, y métodos que nos devuelven, por ejemplo, los pesos.

Vemos que por default inicializa los pesos de forma aleatoria entre 0 y 1, y los bias a cero. Esto se puede cambiar pasando un ``initializer=``al compilar.

In [ ]:
hidden1 = model.layers[1]
print(hidden1.name)
weights, biases = hidden1.get_weights()
print(weights.shape)
print(weights[0], biases[0])


# Entrenar

Entrenar es tan facil como hacer un ``.fit``, donde podemos especificar muchas cosas como épocas, datos de validación (que evaluará al final de cada época), métricas, etc.

El método devuelve un objeto de Historia, con toda la información del entrenamiento (en forma de diccionarios), que podremos utilizar para analizar el modelo.

In [ ]:
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))

Examinemos el diccionario de parámetros del fit:

In [ ]:
history.params

In [ ]:
history.history.keys()

Podemos usar el atributo "history" que nos da un diccionario, para plotear las métricas que usamos.

In [ ]:
import pandas as pd

pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1)
plt.show()

# Evaluar y predecir

Evaluar en un conjunto de Test, o hacer predicciones, es sumamente fácil:

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
X_new = X_test[:3]
y_proba = model.predict(X_new)
y_proba.round(2)

In [ ]:
y_pred = model.predict_classes(X_new)
y_pred

In [ ]:
y_pred = np.argmax(y_proba, axis=-1)
y_pred

In [ ]:
np.array(class_names)[y_pred]

In [ ]:
plt.figure(figsize=(7.2, 2.4))
for index, image in enumerate(X_new):
    plt.subplot(1, 3, index + 1)
    plt.imshow(image, cmap="binary", interpolation="nearest")
    plt.axis('off')
    plt.title(class_names[y_test[index]], fontsize=12)
plt.subplots_adjust(wspace=0.2, hspace=0.5)
plt.show()

# Guardar y Cargar Modelos

Guardar modelos es tan sencillo como hacer

In [ ]:
model.save("my_keras_model.h5")

Que luego se puede cargar (por ejemplo en otra computadora, con otro script, para hacer predicciones) usando:

In [ ]:
model = keras.models.load_model("my_keras_model.h5")

También podemos guardar solamente los weights de los layers. Pero en este caso deberemos, antes de cargarlos, definir el mismo modelo que usamos y recien entonces cargar los weights

In [ ]:
model.save_weights("my_keras_weights.ckpt")

In [ ]:
model.load_weights("my_keras_weights.ckpt")

# Regresión: California Housing

Veamos como se desempeña para hacer regresión con el dataset de california housing que utilizamos en la guía de ejercicios Datasets. Importamos los datos, y los escaleamos usando StandardScaler

In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

El modelo que definimos tiene como ultimo layer una funcion lineal, sin ninguna funcion de activación. (Es como hacer regresión lineal, pero fiteando unidades ReLu como funciones de base).

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=X_train.shape[1:]),
    keras.layers.Dense(1)
])
model.compile(loss="mean_squared_error", optimizer=keras.optimizers.SGD(lr=1e-3))
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))
mse_test = model.evaluate(X_test, y_test)
X_new = X_test[:3]
y_pred = model.predict(X_new)

In [ ]:
pd.DataFrame(history.history).plot()
plt.grid(True)
plt.gca().set_ylim(0, 1)
plt.show()
print(y_pred)

## Ventajas de la API funcional: Modelos no secuenciales

Aqui vemos un ejemplo de como podemos usar la API funcional para hacer modelos mas complejos. En el modelo [Wide and Deep](https://https://ai.google/research/pubs/pub45413) se conectan los inputs directamente a los outputs, ademas de pasando por una red profunda de dos capas. De esta manera, el modelo puede aprender features mas abstractos de los datos, y a la vez tomarlos directamente sin distorisión.

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

input_ = keras.layers.Input(shape=X_train.shape[1:])
hidden1 = keras.layers.Dense(30, activation="relu")(input_)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.concatenate([input_, hidden2])
output = keras.layers.Dense(1)(concat)
model = keras.models.Model(inputs=[input_], outputs=[output])

model.summary()

In [ ]:
keras.utils.plot_model(model, "wide_and_deep_model.png", show_shapes=True)

In [ ]:
model.compile(loss="mean_squared_error", optimizer=keras.optimizers.SGD(lr=1e-3))
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))
mse_test = model.evaluate(X_test, y_test)
y_pred = model.predict(X_new)

# EXTRAS

Callbacks son una manera de hacer cosas durante el entrenamiento. Son utiles en caso que el modelo tarde mucho en fitear. Por ejemplo, podemos guardar el modelo al fin de cada epoca (por si se nos apaga la maquina en el medio del training), o implementar un regularizador EarlyStopping y dejar de fitear si el validation score no mejora, o enviarnos un mail cuando termine el entrenamiento con los resultados, etc.

## Callbacks: Guardar durante entrenamiento (checkpoints)

In [ ]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=[8]),
    keras.layers.Dense(30, activation="relu"),
    keras.layers.Dense(1)
])

model.compile(loss="mse", optimizer=keras.optimizers.SGD(lr=1e-3))

In [ ]:
 checkpoint_cb = keras.callbacks.ModelCheckpoint("my_keras_model.h5", save_best_only=True)

 history = model.fit(X_train, y_train, epochs=30,
                    validation_data=(X_valid, y_valid),
                     callbacks=[checkpoint_cb])

In [ ]:
model = keras.models.load_model("my_keras_model.h5")

In [ ]:
model.evaluate(X_valid,y_valid) 

## Callbacks: Early Stopping

In [ ]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=[8]),
    keras.layers.Dense(30, activation="relu"),
    keras.layers.Dense(1)
])

model.compile(loss="mse", optimizer=keras.optimizers.SGD(lr=1e-3))

In [ ]:
model.compile(loss="mse", optimizer=keras.optimizers.SGD(lr=1e-3))
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10,
                                                  restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=100,
                    validation_data=(X_valid, y_valid),
                    callbacks=[checkpoint_cb, early_stopping_cb])
mse_test = model.evaluate(X_test, y_test)

## Busqueda de Hiperparámetros: Sklearn integration

Y lo mejor de todo, tiene un wrapper que nos permite integrarlo con ScikitLearn. Sklearn lo reconocerá el modelo como uno más de los suyos:

In [ ]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[8]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu"))
    model.add(keras.layers.Dense(1))
    optimizer = keras.optimizers.SGD(lr=learning_rate)
    model.compile(loss="mse", optimizer=optimizer)
    return model

In [ ]:
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)

In [ ]:
keras_reg.fit(X_train, y_train, epochs=30,
              validation_data=(X_valid, y_valid),
              callbacks=[keras.callbacks.EarlyStopping(patience=10)])

El modelo tiene los métodos usuales de un estimador de sklearn

In [ ]:
mse_test = keras_reg.score(X_test, y_test)
y_pred = keras_reg.predict(X_new)

Por ejemplo, podemos usarlo para buscar hiperparámetros con RandomizedSearchCV

In [ ]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV


np.random.seed(42)
tf.random.set_seed(42)

param_distribs = {
    "n_hidden": [0, 1, 2, 3],
    "n_neurons": np.arange(1, 100),
    "learning_rate": reciprocal(3e-4, 3e-2),
}

rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=10, cv=3, verbose=2)
rnd_search_cv.fit(X_train, y_train, epochs=100,
                  validation_data=(X_valid, y_valid),
                  callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Y accedemos a los resultados del CV como siempre hacemos en sklearn:

In [ ]:
rnd_search_cv.best_params_

In [ ]:
rnd_search_cv.best_score_

In [ ]:
rnd_search_cv.best_estimator_

In [ ]:
rnd_search_cv.score(X_test, y_test)

In [ ]:
model = rnd_search_cv.best_estimator_.model
model.evaluate(X_test, y_test)

### Otras librerías para Optimización:

Hay muchas, que hacen cosas un poco mejores que buscar en una grilla o de forma Random. Acá selecciono tres a dedo (en el Gerón pueden ver más opcionaes también):

 * [Keras Tuner](https://keras-team.github.io/keras-tuner/): Es parte del ecosistema de Keras, por lo que está optimizado para la versión de tensorflor: tf.keras, y tiene herramientas de visualización útiles.
 * [Scikit-Optimize](https://scikit-optimize.github.io/): Similar a sklearn para algoritmos de aprendizaje automatizado, es esta librería para algoritmos de optimización. En particular `BayesSearchCV` hace optimización Bayesiana con una interfaz similar al `GridSearchCV`.
 * [Sklearn-Deap](https://github.com/rsteca/sklearn-deap): Esta pretende imitar al `GridSearchCV` de sklearn para buscar hiperparámetros, pero utilizando algoritmos evolutivos en su lugar.

# Regularización

Sabemos como hacer más flexible nuestra red: Aumentamos el numero de Layers o de Neuronas por Layer. Pero, cómo la regularizamos para prevenir overfitting?

Acá presentamos tres formas:

## Weight-decay: Regularización $\ell_2$ ó $\ell_1$.

Similar a los métodos de regularización Ridge y Lasso para modelos lineales, que añadían un término a la función de costo proporcional a la norma $\ell_2$ o $\ell_1$ del vector de pesos, uno puede hacer lo mismo en la red neuronal. 

Implementar weight decay en una red neuronal en keras, es tan simple como añadir el argumento `kernel_regularizer=` en los layers que queramos regularizar. Los regularizadores $\ell_2$ o $\ell_1$ los podemos encontrar en `keras.regularizers.l2()` y  `keras.regularizers.l1()`, o incluso un ElasticNet (combinacion de ambos) en `keras.regularizers.l1_l2()`. Al inicializarlos les deberíamos pasar como argumento el peso que tendrá estos términos (equivalente al `alpha` en sklearn).

In [ ]:
#por ejemplo para definir un layer denso ReLy de 100 neuronas y regularización l2 (~Ridge) con alpha=0.01:
layer = keras.layers.Dense(100,
                          activation='relu',
                          kernel_regularizer=keras.regularizers.l2(0.01))

Gerón ademas nos da un truco para ahorrarnos escribir esto en cada layer, y cualquier hiperparámetro que se repita: Crear un wrapper a dicho layer con la función de Python `functools.partial`. Esta crea wrappers sencillos a funciones/objetos, que pasan hiperparámetros default.

In [ ]:
#Por ejemplo el modelo primero del notebook, pero regularizado:
from functools import partial

RegularizedDense = partial(keras.layers.Dense,
                          activation='relu',
                          kernel_regularizer=keras.regularizers.l2(0.01))

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    RegularizedDense(300),
    RegularizedDense(100),
    RegularizedDense(10, activation="softmax")
])

## Dropout Layer

Una forma muy popular de hacer regularización fue propuesta en dos papers de 2012 y 2014: En cada paso de entrenamiento, algunas neuronas de la red se "apagan" con una probabilidad $p$. Esto solo se hace durante entrenamiento (no durante test o en predicciones). La idea de fondo es que estamos forzando a la red a que no dependa fuertemente de ninguna neurona en particular, sino que sería dar mucha importancia a algún feature en particular, sino que la forzamos a dar resultados aún cuando ciertas neuronas se apagan.

En keras, esto ya ha sido implementado como un Layer, así que solamente lo debemos importar. Este se encargará de eliminar en cada training_step algunas neuronas con un determinado `rate`, retropropagar correctamente el error y adaptar los weights correctamente a la hora de evaluación y predicción.

In [ ]:
#Por ejemplo el modelo primero del notebook, pero con Dropout luego de cada Layer:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dropout(rate=0.2), #esto lo que esta apagando son algunos pixels de la imagen.
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(10, activation="softmax")
])

## Early Stopping

Si bien a medida que entrenamos, el costo sobre el training set va decreciendo, cuando comenzamos a sobreajustarlo típicamente el costo en el validation set comenzará a subir. Por esto, una forma de regularizar es ir monitoreando la función de costo sobre el conjunto de validación al final de cada época, y si vemos que luego de ciertas épocas la validación no mejora, simplemente dejamos de entrenar. 

En keras, early stopping es fácilmente implementable con un Callback, como vimos anteriormente en EXTRAS. Para visualizar a lo que nos referimos, en un tiempo corto, entrenemos la misma red que al principio, pero invirtiendo los roles de entrenamiento y validación. Como el conjunto de validación es pequeño, deberíamos ver que enseguida comenzamos a sobreajustarlo.


In [ ]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=[8]),
    keras.layers.Dense(30, activation="relu"),
    keras.layers.Dense(1)
])

model.compile(loss="mse", optimizer=keras.optimizers.SGD(lr=1e-3))

In [ ]:
#aquí invierto el rol de los datasets
history = model.fit(X_train[:100], y_train[:100], epochs=50,
                    validation_data=(X_valid, y_valid))

Veamos como dan los plots

In [ ]:
import pandas as pd

pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.show()

Claramente será mejor simplemente dejar de entrenar en la época 20. Esto lo hacemos con el callback de EarlyStopping.

In [ ]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=[8]),
    keras.layers.Dense(30, activation="relu"),
    keras.layers.Dense(1)
])

model.compile(loss="mse", optimizer=keras.optimizers.SGD(lr=1e-3))

In [ ]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10,
                                                  restore_best_weights=True)
history = model.fit(X_train[:100], y_train[:100], epochs=50,
                    validation_data=(X_valid, y_valid),
                    callbacks=[early_stopping_cb])

In [ ]:
pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.show()

Vemos que paró en la época 31, porque pusimos una paciencia de 10 épocas (espera 10 épocas a ver si vuelve a mejorar, en caso que sea una fluctuación. Sin embargo, como pusimos `restore_best_weights=True`, el modelo fiteado debería corresponder al de la época 21:

In [ ]:
model.evaluate(X_valid, y_valid, verbose=0)

Que corresponde a la valid_loss en la época 21.

# Vanishing/Exploding Gradients

Debido a la naturaleza misma de la retro-propagacion, los gradientes son muy inestables y las capas pueden aprender a distintas velocidades.

Si los gradientes disminuyen a medida que retrodecemos en la red, las capas mas cercanas al input van a cambiar muchos menos, evitando que la red converga a una solucion. Esto es lo que se llama _vanishing gradient_.

Si los gradientes aumentan mas y mas, el algoritmo empieza a diverger. Es lo que se llama _exploding gradient_.

Suele ser una conspiracion entre la funcion de activacion y la inicializacion de los pesos que causa que haya saturacion en las capas y por ende el gradiente disminuya mas y mas.

In [ ]:
def logit(z):
    return 1 / (1 + np.exp(-z))

z = np.linspace(-5, 5, 200)

plt.plot([-5, 5], [0, 0], 'k-')
plt.plot([-5, 5], [1, 1], 'k--')
plt.plot([0, 0], [-0.2, 1.2], 'k-')
plt.plot([-5, 5], [-3/4, 7/4], 'g--')
plt.plot(z, logit(z), "b-", linewidth=2)
props = dict(facecolor='black', shrink=0.1)
plt.annotate('Saturating', xytext=(3.5, 0.7), xy=(5, 1), arrowprops=props, fontsize=14, ha="center")
plt.annotate('Saturating', xytext=(-3.5, 0.3), xy=(-5, 0), arrowprops=props, fontsize=14, ha="center")
plt.annotate('Linear', xytext=(2, 0.2), xy=(0, 0.5), arrowprops=props, fontsize=14, ha="center")
plt.grid(True)
plt.title("Sigmoid activation function", fontsize=14)
plt.axis([-5, 5, -0.2, 1.2])
plt.show()

## Inicializacion de los pesos

Una manera de evitar que esto ocurra es inicializar los pesos de manera tal que sucedan dos cosas: que las varianzas de los inputs y outputs de cada neurona sean iguales entre si, y que la varianza del gradiente calculada en la pasada forward y en la backward coincidan.

No se pueden garantizar ambas si la capa no tiene igual cantidad de inputs que de outputs pero puede llegarse a un buen compromiso.

Aparecen entonces distintas estrategias de inicializacion, a veces asociadas a determinadas funciones de activacion.

In [ ]:
[name for name in dir(keras.initializers) if not name.startswith("_")]

Y esto se incorpora de esta manera:

In [ ]:
keras.layers.Dense(10, activation="relu", kernel_initializer="he_normal")

O uno propio con:

In [ ]:
init = keras.initializers.VarianceScaling(scale=2., mode='fan_avg',
                                          distribution='uniform')
keras.layers.Dense(10, activation="relu", kernel_initializer=init)

## Dying Relus

Las ReLUs mueren. LeakyRelUs, RReLUs, etc Funcionan mejor pero overfittean.

ELU, SELU: funcionan mejor, no tienen los problemas de las ReLUs pero son mas lentas a la hora de evaluar.

In [ ]:
def leaky_relu(z, alpha=0.01):
    return np.maximum(alpha*z, z)

plt.plot(z, leaky_relu(z, 0.05), "b-", linewidth=2)
plt.plot([-5, 5], [0, 0], 'k-')
plt.plot([0, 0], [-0.5, 4.2], 'k-')
plt.grid(True)
props = dict(facecolor='black', shrink=0.1)
plt.annotate('Leak', xytext=(-3.5, 0.5), xy=(-5, -0.2), arrowprops=props, fontsize=14, ha="center")
plt.title("Leaky ReLU activation function", fontsize=14)
plt.axis([-5, 5, -0.5, 4.2])

plt.show()

In [ ]:
[m for m in dir(keras.activations) if not m.startswith("_")]

In [ ]:
[m for m in dir(keras.layers) if "relu" in m.lower()]


## Batch Normalization

## GradientClipping

In [ ]:
optimizer = keras.optimizers.SGD(clipvalue=1.0)

# Optimizadores